In [2]:
import pandas as pd
import numpy as np
from google.colab import drive


# SIMULAR DATOS CLIMÁTICOS HISTÓRICOS (Sintéticos)
# * **Datos Climáticos:** Me basé en datos del **Servicio Meteorológico Nacional (SMN)**
#     y en datos satelitales de precipitación como los de **CHIRPS**.https://www.chc.ucsb.edu/
# * **Datos Agrícolas:** Utilicé estimaciones de rendimientos del **Ministerio de
#     Agricultura, Ganadería y Pesca de la Nación (MAGyP)** para crear las relaciones
#     realistas entre clima y rendimiento. https://datosestimaciones.magyp.gob.ar/reportes.php?reporte=Estimaciones
np.random.seed(42) # Para que los resultados sean siempre los mismos

# Creamos un DataFrame con 10 años de datos climáticos simulados
num_anios = 10
anios = np.arange(2015, 2015 + num_anios)

# Simula la lluvia acumulada y la temperatura media
lluvia_anual_simulada = np.random.randint(400, 800, size=num_anios)
temp_media_anual_simulada = np.random.uniform(20, 25, size=num_anios)

# Creamos el DataFrame de features climáticas
df_features_anuales = pd.DataFrame({
    'YEAR': anios,
    'lluvia_acumulada_anual': lluvia_anual_simulada,
    'temp_media_anual': temp_media_anual_simulada
})

# PASO 2: SIMULAR DATOS DE RENDIMIENTO HISTÓRICO (Sintéticos)
# El rendimiento tiende a ser mayor con más lluvia y menor con menos lluvia.
maiz_rendimiento_base = 6500
soja_rendimiento_base = 2500

# Añadi un poco de variabilidad aleatoria
maiz_simulado = maiz_rendimiento_base + (lluvia_anual_simulada - 600) * 5 + np.random.randint(-200, 200, num_anios)
soja_simulada = soja_rendimiento_base + (lluvia_anual_simulada - 600) * 2 + np.random.randint(-100, 100, num_anios)

df_rendimientos_pivot = pd.DataFrame({
    'YEAR': anios,
    'Maíz': maiz_simulado,
    'Soja total': soja_simulada
})


#FUSIÓN DE LOS DATAFRAMES

# Unimos los features de clima con los rendimientos por el año
df_final_entrenamiento = pd.merge(df_features_anuales, df_rendimientos_pivot, on='YEAR', how='inner')

# RESULTADO FINAL

print("DataFrame final listo para el entrenamiento del modelo:")
print(df_final_entrenamiento)


DataFrame final listo para el entrenamiento del modelo:
   YEAR  lluvia_acumulada_anual  temp_media_anual  Maíz  Soja total
0  2015                     502         20.290418  6123        2393
1  2016                     748         24.330881  7061        2885
2  2017                     670         23.005575  6902        2714
3  2018                     506         23.540363  6065        2401
4  2019                     471         20.102922  5999        2192
5  2020                     588         24.849549  6288        2483
6  2021                     420         24.162213  5458        2094
7  2022                     502         21.061696  5979        2267
8  2023                     521         20.909125  6092        2372
9  2024                     614         20.917023  6640        2478


In [3]:
print("Información del DataFrame final (df.info()):")

df_final_entrenamiento.info()

print("Descripción estadística del DataFrame final (df.describe()):")

df_final_entrenamiento.describe()

Información del DataFrame final (df.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   YEAR                    10 non-null     int64  
 1   lluvia_acumulada_anual  10 non-null     int64  
 2   temp_media_anual        10 non-null     float64
 3   Maíz                    10 non-null     int64  
 4   Soja total              10 non-null     int64  
dtypes: float64(1), int64(4)
memory usage: 532.0 bytes
Descripción estadística del DataFrame final (df.describe()):


,YEAR,lluvia_acumulada_anual,temp_media_anual,Maíz,Soja total
count,10.00000,10.00000,10.000000,10.000000,10.000000
mean,2019.50000,554.20000,22.316976,6260.700000,2427.900000
std,3.02765,99.71827,1.837030,480.075006,234.386409
min,2015.00000,420.00000,20.102922,5458.000000,2094.000000
25%,2017.25000,502.00000,20.911099,6015.500000,2293.250000
50%,2019.50000,513.50000,22.033635,6107.500000,2397.000000
75%,2021.75000,607.50000,24.006751,6552.000000,2481.750000
max,2024.00000,748.00000,24.849549,7061.000000,2885.000000


df.info() te dice que tienes 10 filas de datos (10 años), sin valores nulos. Todas tus columnas son numéricas,

df.describe() el resumen estadístico se puedes ver que:

La lluvia promedio en esos 10 años fue de 554.2 mm.

El rendimiento promedio de maíz fue de 6,260.7 kg/ha, con una desviación estándar (std) de 480.07 kg/ha, lo que indica que el rendimiento varía significativamente.

In [5]:
#  AÑADIR LA VARIABLE DE PRONÓSTICO. Fuente Local: Servicio Meteorológico Nacional (SMN) de Argentina
# Con la información histórica de la NOAA, añadimos la columna estado_enso.

df_final_entrenamiento['estado_enso'] = [1, -1, 0, 1, 0, -1, -1, -1, 1, 0]

# Muestra el DataFrame actualizado para verificar
print(df_final_entrenamiento)


   YEAR  lluvia_acumulada_anual  temp_media_anual  Maíz  Soja total  \
0  2015                     502         20.290418  6123        2393   
1  2016                     748         24.330881  7061        2885   
2  2017                     670         23.005575  6902        2714   
3  2018                     506         23.540363  6065        2401   
4  2019                     471         20.102922  5999        2192   
5  2020                     588         24.849549  6288        2483   
6  2021                     420         24.162213  5458        2094   
7  2022                     502         21.061696  5979        2267   
8  2023                     521         20.909125  6092        2372   
9  2024                     614         20.917023  6640        2478   

   estado_enso  
0            1  
1           -1  
2            0  
3            1  
4            0  
5           -1  
6           -1  
7           -1  
8            1  
9            0  


In [6]:
from google.colab import drive
drive.mount('/content/drive')
ruta_guardado = '/content/drive/MyDrive/df_entrenamiento_sintetico.csv'
df_final_entrenamiento.to_csv(ruta_guardado, index=False)

print("\nDataFrame final listo para el entrenamiento del modelo:")
print(df_final_entrenamiento)
print(f"\nDataFrame guardado con éxito en: {ruta_guardado}")

Mounted at /content/drive

DataFrame final listo para el entrenamiento del modelo:
   YEAR  lluvia_acumulada_anual  temp_media_anual  Maíz  Soja total  \
0  2015                     502         20.290418  6123        2393   
1  2016                     748         24.330881  7061        2885   
2  2017                     670         23.005575  6902        2714   
3  2018                     506         23.540363  6065        2401   
4  2019                     471         20.102922  5999        2192   
5  2020                     588         24.849549  6288        2483   
6  2021                     420         24.162213  5458        2094   
7  2022                     502         21.061696  5979        2267   
8  2023                     521         20.909125  6092        2372   
9  2024                     614         20.917023  6640        2478   

   estado_enso  
0            1  
1           -1  
2            0  
3            1  
4            0  
5           -1  
6           -1  

In [ ]:
print(df_final_entrenamiento.head())